# Generating german doctor reviews with a GPT-2 model
## Fine tuning of a pretrained **Hugging Face** transfomer decoder
In this notebook we will be using a GPT-2 mdoel that was fine-tuned to synthesize doctor reviews mimiking actual patients' text comments.

A detailed description of the **German language reviews of doctors by patients 2019** dataset can be found [here](https://data.world/mc51/german-language-reviews-of-doctors-by-patients)


For this exercise, we will use the [**Hugging Face**](https://huggingface.co/) implementation of transformers for Tensorflow 2.0. Transformers provides a general architecture implementation for several state of the art models in the natural language domain.

NOTE: This notebook and its implementation is heavily influenced by the [data-drive](https://data-dive.com/) *Natural Language Processing of German texts* blog post

In [1]:
!pip install -U transformers==4.9.2

     |████████████████████████████████| 2.6 MB 26.0 MB/s 
     |████████████████████████████████| 3.3 MB 30.4 MB/s 
     |████████████████████████████████| 895 kB 56.7 MB/s 
     |████████████████████████████████| 636 kB 58.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import pandas as pd
import tensorflow as tf

from transformers import AutoTokenizer, TFGPT2LMHeadModel

pd.options.display.max_colwidth = 600
pd.options.display.max_rows = 400

## Setting up the decoder model
HuggingFace's transfomer library allows for conviniently loading  pre-configured text tokenizers and pre-trained models from local resources.

Here we will be using a tokenizer and a GPT-2 model that was pre-trained on the doctor review dataset


In [3]:
!rm -r gpt2_doctorreview_finetuned* __MACOSX
!gdown https://drive.google.com/uc?id=13wbf5bsLmvRFD-AgbmruWo6bdiyjkwd9 -O gpt2_doctorreview_finetuned.zip
!unzip gpt2_doctorreview_finetuned.zip

--2021-09-20 14:16:30--  https://github.com/AdvancedNLP/decoder/raw/main/gpt2_doctorreview_finetuned.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/AdvancedNLP/decoder/main/gpt2_doctorreview_finetuned.zip [following]
--2021-09-20 14:16:30--  https://media.githubusercontent.com/media/AdvancedNLP/decoder/main/gpt2_doctorreview_finetuned.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 462732204 (441M) [application/zip]
Saving to: ‘gpt2_doctorreview_finetuned.zip’

gpt2_doctorreview_f 100%[===================>] 441.29M   163MB/s    in 2.7s    

2021-09-20 14:16:44 (163 MB/s)

In [4]:
tokenizer = AutoTokenizer.from_pretrained('gpt2_doctorreview_finetuned/tokenizer')
model = TFGPT2LMHeadModel.from_pretrained('gpt2_doctorreview_finetuned/model')

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2_doctorreview_finetuned/model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


## Generating doctor reviews
The model has been conditioned to be able to control if positive or negative reviews should be generated. 

As an auto-regressive model the sequence is generated by building up from the passed input sequences. We can use this to control the polarity of the review by passing either the token for positive or for negative reviews

In [5]:
POS_TOKEN = "<|review_pos|>"
NEG_TOKEN = "<|review_neg|>"

### Simple greedy search
Let's implement our own greedy-search-based text generator. Generation happens in a loop where one token is generated at a time. Token with highest probability is select in each iteration.


In [6]:
def generate_greedy(inputs:str, max_length=15):
    #print('Input: ', inputs)
    input_ids = tokenizer.encode(inputs, return_tensors='tf')

    for __ in range(max_length):
        
        logits = model.predict(input_ids).logits

        ##########################
        ## YOUR CODE HERE START ##
        ##########################

        # retrieve the predicted logits for the *last* token
        # Dimensions are [batch_size, input_tokens, vocab_size]
        next_token_logits = logits[:, -1, :]  

        # Select the token with the highest probability and convert it to tf.int32
        next_token = tf.math.argmax(next_token_logits, axis=-1, output_type=tf.int32)

        # Concat the previous tokens with the new one
        # You will have to expand the dimension of the next token to match 
        # the shape of input_ids
        input_ids = tf.concat([input_ids, tf.expand_dims(next_token, -1)], 1)

    output_ids = input_ids.numpy().squeeze()

    ### Use your tokenizer to convert the output ids into text
    decoded = tokenizer.decode(output_ids)

    ##########################
    ## YOUR CODE HERE END ##
    ##########################
    return decoded

In [14]:
generate_greedy(POS_TOKEN + ' Ich', max_length=15)

'<|review_pos|> Ich bin seit Jahren bei Dr. Heuer in Behandlung und bin sehr zufrieden.'

### More advanced text generation
So far so good. Now we understand how text can be generated.

However we ignore when our model predicts EOS (end-of-sentence). What would be neccessary to incoorporate this in our function?

What if we would want to generate multiple different review comments?
Did you generate long reviews? Have you started to see repetitions in the generated output? Why is that?

Luckily the Hugging Face implementation offers various ways for us to generate higher quality reviews.

#### Greedy search
The following code can be used to generate text using a greedy search algorithm:

In [8]:
# encode context the generation is conditioned on
input_ids = tokenizer.encode(POS_TOKEN, return_tensors='tf')

# generate text until the output length
# (which includes the context length) reaches 50 
greedy_outputs = model.generate(
    input_ids, 
    max_length=50,
    num_return_sequences=3,
    )

genrated_reviews = [{'generated_text': tokenizer.decode(output, skip_special_tokens=True)}
                    for output in greedy_outputs]
pd.DataFrame(genrated_reviews)

,generated_text
0,Wir haben uns während unserer Kinderwunschbehandlung sehr gut aufgehoben gefühlt Die Betreuung durch Dr. Kempkensteffen war ausgezeichnet und sein liebes Team arbeitet Hand in Hand aufeinander abgestimmt und engagiert. Wir können ihn bestens weiterempfehlen und würden Ihn jedem weiter
1,"Ich bin seit Jahre zufriedene Patientin von Frau Dr. Sünter. Sie ist eine sehr kompetente Ärztin, die sich Zeit für ihre Patienten nimmt."
2,Die Wartezeit betrug ca. Minuten für einen Termin. Der Arzt hat sich Zeit genommen und alle Fragen beantwortet. Die Behandlung habe ich als sehr kompetent empfunden.


#### Beam search 
Beam search can be considered as an alternative. At each step of generating a token, a set of top probability tokens are kept as part of the beam instead of just the highest-probability token. The sequence with the highest overall probability is returned at the end of the generation.

What do the parameters `no_repeat_ngram_size` and `temperature` control?

Generating text using beam search is done like this:

In [9]:
beam_outputs = model.generate(
    input_ids,
    max_length=50,
    num_beams=7,
    no_repeat_ngram_size=3,
    num_return_sequences=3,
    early_stopping=True,
    temperature=0.7
)

genrated_reviews = [{'generated_text': tokenizer.decode(output, skip_special_tokens=True)}
                    for output in beam_outputs]
pd.DataFrame(genrated_reviews)

,generated_text
0,Ich bin seit Jahren bei Dr. Heuer in Behandlung und bin sehr zufrieden. Er ist sehr kompetent und nimmt sich Zeit für seine Patienten. Ich kann ihn nur weiterempfehlen.
1,Ich bin seit Jahren bei Frau Dr. Henze und bin sehr zufrieden. Sie ist sehr kompetent und nimmt sich Zeit für ihre Patienten. Ich fühle mich bei ihr sehr gut aufgehoben.
2,Ich bin seit Jahren bei Frau Dr. Henze und bin sehr zufrieden. Sie ist sehr kompetent und nimmt sich Zeit für ihre Patienten. Ich fühle mich bei ihr sehr gut aufgehoben.


#### High level pipeline
The easiest way to to use the model is to use HuggingFaces transformer `pipeline` implementation to encapsulate the previously loaded `model` and `tokenizer`.

The documentation for the [**pipeline**](https://huggingface.co/transformers/main_classes/pipelines.html) abstraction describes how to do the setup.

While being able to generate reviews with very high fiddelity, it's also the slowest approach. Can you find out why?


In [10]:
from transformers import pipeline

In [11]:
##########################
## YOUR CODE HERE START ##
##########################
# build a transformer-pipeline 
# to generate text using the 
# previously loaded model and tokenizer

review_generator = pipeline(
  "text-generation",
  model=model,
  tokenizer=tokenizer,
)

##########################
## YOUR CODE HERE END   ##
##########################

In [12]:
pos_generated_reviews = review_generator(POS_TOKEN, max_length=50, num_return_sequences=3)
pd.DataFrame(pos_generated_reviews)

,generated_text
0,<|review_pos|> Herr Dr. Plöger hat mich äußerst freundlich und kompetent behandelt. Er hat sich viel Zeit für mein Anliegen genommen und auch eine individuelle und umfassende Beratung durchgeführt. Ich hatte gleich ein gutes Gefühl. Ich kann diese Praxis zu weiterempfehlen.
1,<|review_pos|> Seit Jahren bin ich bei Herrn Dr. Schwarz in Behandlung und kann ihn nur wärmstens weiterempfehlen. Sein Team ist kompetent und immer freundlich.
2,"<|review_pos|> Herr Dr. Runge war immer der mein Hausarzt. Sehr zuvorkommend, nett, hilfsbereit und freundlich. Ich kann ihn mit bestem Gewissen empfehlen"


In [13]:
neg_generated_reviews = review_generator(NEG_TOKEN, max_length=50, num_return_sequences=3)
pd.DataFrame(neg_generated_reviews)

,generated_text
0,<|review_neg|> Ich werde nicht mit meinem Kind zum Arzt gehen !!
1,"<|review_neg|> Ich war bei dieser Ärztin ein Jahr . Sie ist sehr freundlich und nimmt sich Zeit für ihre Patienten. In meinen Augen war sie inkompetent, überlässt einem das Gefühl nicht zufrieden zu sein und hat mir keine Zeit gegeben mich zu untersuchen. Die"
2,"<|review_neg|> Ich war zur Untersuchung dort und wollte mir eine zweite Meinung einholen. Er hatte eine sehr hohe Anzahl an Bewertungen und dadurch hatte ich mich für eine Praxis entschieden, bei der ich die Praxis war. Ich habe ein paar Minuten mit dem Arzt verbracht,"
